# Gráficos interactivos con bokeh y widgets

Para lograr gráficas con mayor interactividad recomiendo utilizar la librería [bokeh](https://docs.bokeh.org/en/latest/index.html)

En esta lección veremos algunos ejemplos avanzados con bokeh

## Inspección de datos interactiva

En el siguiente ejemplo se muestra una gráfica básica con bokeh 

Los controles de la derecha permiten mover y hacer zoom sobre los datos

El `tooltip` permite ver el valor del dato cuando el cursor se posa sobre el mismo

In [ ]:
import numpy as np
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

data = np.random.randn(100, 2)
source = ColumnDataSource(data=dict(index=range(100), x=data[:,0], y=data[:,1]))

plot = Figure(plot_width=550, plot_height=300, tooltips=[("index","$index"),("(x,y)", "($x, $y)")])
plot.scatter('x', 'y', size=10, source=source)

show(plot)

## Manipulación de datos usando widgets 

Un widget es una interfaz gráfica de usuario. En el siguiente ejemplo se enlanzan tres controles de desplazamiento a un gráfico de bokeh que es modificado con una rutina escrita en `javascript`

In [ ]:
t = np.linspace(0, 5, num=1000)
s = np.cos(2.0*np.pi*t)

A = Slider(start=0.1, end=2, value=1, step=.01, title="Amplitud")
f = Slider(start=0.1, end=2, value=1, step=.01, title="Frecuencia")
p = Slider(start=0, end=6.4, value=0, step=.1, title="Desfase")

source = ColumnDataSource(data=dict(t=t, s=s))
plot = Figure(y_range=(-1.5, 1.5), plot_width=550, plot_height=300)
plot.line('t', 's', source=source, line_width=3, line_alpha=0.6)
plot.xaxis[0].axis_label = 'Tiempo [s]'

callback = CustomJS(args=dict(source=source, A=A, f=f, p=p), code="""
    var t = source.data['t'];
    var s = source.data['s'];
    for (var i = 0; i < s.length; i++) {
        s[i] = A.value*Math.cos(2*Math.PI*t[i]*f.value + p.value);
    }
    source.change.emit();
""")

for widget in [A, f, p]:
    widget.js_on_change('value', callback)

show(column(A, f, p, plot))